In [1]:
users = 1 # number of clients

In [2]:
import os
import struct
import socket
import pickle
import time

import h5py
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

/Applications/anaconda3/envs/keras/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

# Get the current working directory
root_path = os.getcwd()

# Print the current working directory
print("The root path of this Jupyter notebook is:", root_path)


The root path of this Jupyter notebook is: /Users/tej


In [4]:
root_path = '/Users/tej/models/cifar10_data'


In [5]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "cpu"
torch.manual_seed(777)
if device =="cuda:0":
    torch.cuda.manual_seed_all(777)

In [6]:
client_order = int(input("client_order(start from 0): "))

client_order(start from 0): 0


In [7]:
num_traindata = 50000 // users

In [8]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

from torch.utils.data import Subset


indices = list(range(50000))

part_tr = indices[num_traindata * client_order : num_traindata * (client_order + 1)]


In [9]:
trainset = torchvision.datasets.CIFAR10 (root=root_path, train=True, download=True, transform=transform)

trainset_sub = Subset(trainset, part_tr)

train_loader = torch.utils.data.DataLoader(trainset_sub, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10 (root=root_path, train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
x_train, y_train = next(iter(train_loader))
print(x_train.size())
print(y_train.size())

torch.Size([4, 3, 32, 32])
torch.Size([4])


In [11]:
total_batch = len(train_loader)
print(total_batch)

12500


In [12]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov  1 14:23:31 2018
@author: tshzzz
"""

import torch
import torch.nn as nn


def conv_dw_client(inplane,stride=1):
    return nn.Sequential(
        nn.Conv2d(inplane,inplane,kernel_size = 3,groups = inplane,stride=stride,padding=1),
        nn.BatchNorm2d(inplane),
        nn.ReLU()  
        )

def conv_bw(inplane,outplane,kernel_size = 3,stride=1):
    return nn.Sequential(
        nn.Conv2d(inplane,outplane,kernel_size = kernel_size,groups = 1,stride=stride,padding=1),
        nn.BatchNorm2d(outplane),
        nn.ReLU() 
        )


class MobileNet(nn.Module):
    
    def __init__(self,num_class=10):
        super(MobileNet,self).__init__()
        
        layers = []
        layers.append(conv_bw(3,32,3,1))
        layers.append(conv_dw_client(32,1))
#         layers.append(conv_dw(64,128,2))
#         layers.append(conv_dw(128,128,1))
#         layers.append(conv_dw(128,256,2))
#         layers.append(conv_dw(256,256,1))
#         layers.append(conv_dw(256,512,2))

#         for i in range(5):
#             layers.append(conv_dw(512,512,1))
#         layers.append(conv_dw(512,1024,2))
#         layers.append(conv_dw(1024,1024,1))

#         self.classifer = nn.Sequential(
#                 nn.Dropout(0.5),
#                 nn.Linear(1024,num_class)
#                 )
        self.feature = nn.Sequential(*layers)
        
        

    def forward(self,x):
        out = self.feature(x)
#         out = out.mean(3).mean(2)
#         out = out.view(-1,1024)
#         out = self.classifer(out)
        return out

In [13]:
mobilenet_client = MobileNet().to(device)
print(mobilenet_client)

MobileNet(
  (feature): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
)


In [14]:
# from torchsummary import summary

# summary(mobilenet_client, (3, 32, 32))

In [15]:
epoch = 20  # default
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mobilenet_client.parameters(), lr=lr, momentum=0.9)

In [16]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = pickle.dumps(msg)
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    msg =  recvall(sock, msglen)
    msg = pickle.loads(msg)
    return msg

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

In [17]:
host = input("IP address: ")
port = 10080

IP address: 10.0.0.94


In [18]:
start_time = time.time()    # store start time
print("timmer start!")

timmer start!


In [19]:
s = socket.socket()
s.connect((host, port))

In [20]:
epoch = recv_msg(s)   # get epoch
msg = total_batch
send_msg(s, msg)   # send total_batch of train dataset

In [21]:
for e in range(epoch):
    client_weights = recv_msg(s)
    mobilenet_client.load_state_dict(client_weights)
    mobilenet_client.eval()
    for i, data in enumerate(tqdm(train_loader, ncols=100, desc='Epoch '+str(e+1))):
        x, label = data
        x = x.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        output = mobilenet_client(x)
        client_output = output.clone().detach().requires_grad_(True)
        msg = {
            'client_output': client_output,
            'label': label
        }
        send_msg(s, msg)
        client_grad = recv_msg(s)
        output.backward(client_grad)
        optimizer.step()
    send_msg(s, mobilenet_client.state_dict())

Epoch 1: 100%|██████████████████████████████████████████████| 12500/12500 [1:21:31<00:00,  2.56it/s]


In [22]:
end_time = time.time()  #store end time
print("WorkingTime of ",device ,": {} sec".format(end_time - start_time))

WorkingTime of  cpu : 5001.382312774658 sec
